<a href="https://colab.research.google.com/github/hpt09/ResumeParser/blob/master/ExpExtractionSnorkelSpacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snorkel
!pip install nltk
!pip install spacy
!pip install phonenumbers

     |████████████████████████████████| 153kB 2.7MB/s 
     |████████████████████████████████| 3.8MB 8.6MB/s 
     |████████████████████████████████| 1.8MB 34.7MB/s 
     |████████████████████████████████| 6.7MB 38.2MB/s 
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556409 sha256=d52e10d78d1c0944d27f98a9acb1dcf979f2470eea2c455d721bccf5ac772dd8
  Stored in directory: /root/.cache/pip/wheels/de/63/64/3699be2a9d0ccdb37c7f16329acf3863fd76eda58c39c737af
Successfully built networkx
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: networkx 2.5
    Uninstalling networkx-2.5:
      Successfully uninstalled netwo

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from snorkel.labeling import labeling_function
import nltk
import spacy
import phonenumbers

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Information Extraction

In [5]:
import pandas as pd
from snorkel.labeling import labeling_function
import nltk

import spacy
nlp = spacy.load("en_core_web_sm")

In [6]:
df_spacy = pd.read_csv(r"cleaned_sentences.csv", encoding= 'unicode_escape')

In [7]:
ABSTAIN = -1
NAME = 1
SKILL = 2
EMAIL = 3
PHONE = 4
EXP = 5

In [8]:
import re
@labeling_function()
def lf_phone_extraction(x):
    x = x.to_string()
    text = x.lower()
    pattern = r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]'    
    match = re.findall(pattern, text)
    if(len(match)> 0):
        print("Found Phone: ", match, " In: ",text)
        return PHONE
    else:
        return ABSTAIN

In [9]:
import json
import spacy
# exps = ["experience", "year", "month", "1" , "2" , "3", "4", "5", "6", "7", "8", "9"]
exps = ["year", "month", "day"]
experience_array = []

@labeling_function()
def lf_skill_extraction(x):
    x = x.to_string()
    text = x.lower()
    text = text.strip()
    obj = None
    with open('cleaned_skills.json', 'r') as skills:
        data = skills.read()
        obj = json.loads(data)
    for skill in obj:
        skill = skill.lower()
 #       skill = (" " + skill + " ")
        for word in text.split():
          if(skill == word):
              print("Found Skill: ", skill, " In: ",text)
              nlp = spacy.load("en_core_web_sm")
              doc = nlp(x)
              for ent in doc.ents:
                if(ent.label_ == 'DATE'):
                  experience_array.append(text)
                  return EXP 
              return SKILL
    return ABSTAIN

@labeling_function()
def lf_temp1(x):
    return ABSTAIN
    
@labeling_function()
def lf_temp2(x):
    return ABSTAIN

In [10]:
import re
@labeling_function()
def lf_email_extraction(x):
    x = x.to_string()
    ners = nlp(x)
    text = x.lower()
    pattern = '[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}'
    match = re.findall(pattern, text)
    if(len(match)> 0):
        print("Found Email: ", match, " In: ",text)
        return EMAIL
    else:
        return ABSTAIN

In [11]:
# TODO: Work on getting less false positives
from spacy.matcher import Matcher
@labeling_function()
def lf_name_extraction(x):
    x = str(x)
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)
    nlp_text = nlp(x)
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    matcher.add('NAME', None, pattern)
    matches = matcher(nlp_text)
    if len(matches)>0:
        for match_id, start, end in matches:
            span = nlp_text[start:end]
            print("Found Name: ", span)
        return NAME
           
    else:
        return ABSTAIN

In [12]:
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel

In [13]:
lfs = [lf_skill_extraction, lf_temp1, lf_temp2]

In [14]:
applier = PandasLFApplier(lfs)

In [15]:
L_train = applier.apply(df_spacy)

  0%|          | 0/69516 [00:00<?, ?it/s]

Found Skill:  c  In:  content    a bright, talented and ambitious c developer w...


  0%|          | 10/69516 [00:00<1:10:38, 16.40it/s]

Found Skill:  c  In:  content    c developer june 2008 - present


  0%|          | 11/69516 [00:01<4:35:25,  4.21it/s]

...

100%|█████████▉| 69268/69516 [26:30<00:11, 20.82it/s]

Found Skill:  excel  In:  content     enthusiasm and a desire to excel


100%|█████████▉| 69420/69516 [26:30<00:02, 38.42it/s]

Found Skill:  clean  In:  content     regulate the process during batch and clean i...
Found Skill:  processing  In:  content     upstream processing of h1n1 monovalent vaccin...


100%|██████████| 69516/69516 [26:32<00:00, 43.66it/s]


In [16]:
label_model = LabelModel(cardinality=6, verbose=True)

In [17]:
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)

In [18]:
df_spacy["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

In [19]:
df_spacy["label"]

0       -1
1       -1
2       -1
3       -1
4       -1
        ..
69511   -1
69512   -1
69513   -1
69514   -1
69515   -1
Name: label, Length: 69516, dtype: int64

In [20]:
df_spacy.to_csv(path_or_buf = 'snorkel_labeled.csv',index=False)
df_spacy = df_spacy[df_spacy.label != ABSTAIN]

In [24]:
exp_array = []
for e in range(len(experience_array)):
  exp_array.append(experience_array[e][12:])
  print(experience_array[e])



In [23]:
import csv

with open('Experience.csv','w') as file:
  file.write('Experience')
  file.write('\n\n')

  for e in exp_array:
      file.write(e)
      file.write('\n')